## Shift detection methods (work in progress)

This tutorial will go into detail about shift detection methods and how to implement your own.

In [1]:
# Prerequisites
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 300
plt.rcParams["figure.figsize"] = (12, 5)

# Include this once we have a published release to fetch test data
# from toad.utils import download_test_data
# download_test_data()

In [2]:
from toad import TOAD

td = TOAD(xr.open_dataset("test_data/garbe_2020_antarctica.nc").coarsen(x=6, y=6, time=5, boundary='trim').mean())